## Preprocesamiento - Titanic (Kaggle)

### Objetivos
* Imputar o eliminar valores faltantes según criterio justificado.
* Codificar variables categóricas (OneHot, LabelEncoder, etc.).
* Normalizar o estandarizar variables numéricas.
* Analizar si aplicar PCA u otra técnica de reducción de dimensión mejora el modelo.

Importación de librerías

In [21]:
#Importación de librerías

#%pip install pandas numpy matplotlib seaborn scipy   

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline
sns.set_theme(style="darkgrid")

### Carga del dataset de entrenamiento

In [22]:
# Carga del dataset de entrenamiento
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Creación de columnas derivadas

In [23]:
extended_df = df.copy()

# Tamaño de familia - FamilySize
extended_df["FamilySize"] = extended_df["SibSp"] + extended_df["Parch"] + 1

# Está sólo - IsAlone
extended_df["IsAlone"] = (extended_df["FamilySize"] == 1).astype(int)

extended_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1


### Limpieza de datos - Eliminación de columnas

* *PassengerId*: No aporta información relevante.
* *Name*: El nombre de la persona no aporta información sobre la posibilidad de supervivencia. Por otro lado, de esta columna se podría extraer el título de la persona (Mr, Mrs, Miss, etc.) y tratar de encontrar una relación entre este tratamiento influye en el nivel de supervivencia. Sin embargo, a los fines de simplificar este trabajo esto no será tenido en cuenta.
* *Cabin*: No es demasiado relevante, además tiene la mayoria de valores nulos.
* *Ticket*: Dado que no se sabe si el ticket correspondía alguna ubicación determinada en el barco, este campo tampoco será tenido en cuenta.

Por todo esto es que se descartarán las columnas PassengerId, Name, Ticket y Cabin.

In [24]:
reduced_df = extended_df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         714 non-null    float64
 4   SibSp       891 non-null    int64  
 5   Parch       891 non-null    int64  
 6   Fare        891 non-null    float64
 7   Embarked    889 non-null    object 
 8   FamilySize  891 non-null    int64  
 9   IsAlone     891 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 69.7+ KB


#### Imputación de datos faltantes

**Age**<br/>Para los datos faltantes de edad (Age) se utilizará el método de imputación simple Hot-Deck ó KNN dado que:

* Usa toda la información disponible del pasajero.
* Mantiene la correlación entre edad y otras variables (clase, sexo, tarifa, etc.).
* Produce valores coherentes con cada individuo.

**Embarked**<br/>Para los datos faltantes de Embarked se utilizará la imputación por la mediana dado que:
* Embarked es una variable categórica con solo 3 categorías.
* Solo hay 2 valores faltantes en el dataset original.
* La distribución es muy desbalanceada: S ~ 72%, C ~ 19%, Q ~ 9%


Puerto de embarque (Embarked) - Imputación por mediana 

In [25]:
completed_df = reduced_df.copy()

completed_df["Embarked"] = completed_df["Embarked"].fillna(completed_df["Embarked"].mode().iloc[0])

completed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         714 non-null    float64
 4   SibSp       891 non-null    int64  
 5   Parch       891 non-null    int64  
 6   Fare        891 non-null    float64
 7   Embarked    891 non-null    object 
 8   FamilySize  891 non-null    int64  
 9   IsAlone     891 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 69.7+ KB


Edad (Age) - Imputación simple KNN

In [26]:
from sklearn.impute import KNNImputer

CARACTERISTICAS_A_IMPUTAR = ["Age", "Pclass", "SibSp", "Parch", "Fare"]

def impute_age_with_knn(dataframe, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    features = dataframe[CARACTERISTICAS_A_IMPUTAR]
    imputed_array = imputer.fit_transform(features)
    dataframe = dataframe.copy()
    dataframe["Age"] = imputed_array[:, 0]
    return dataframe

completed_df = impute_age_with_knn(completed_df)

completed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         891 non-null    float64
 4   SibSp       891 non-null    int64  
 5   Parch       891 non-null    int64  
 6   Fare        891 non-null    float64
 7   Embarked    891 non-null    object 
 8   FamilySize  891 non-null    int64  
 9   IsAlone     891 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 69.7+ KB


A continuación de muestra una comparación entre las estadísiticas descriptivas edad originales e imputada.

In [27]:
print("\nEstadísticas descriptivas de los datos originales de edad:")
df["Age"].describe()


Estadísticas descriptivas de los datos originales de edad:


count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [28]:
print("\nEstadísticas descriptivas de los datos completados de edad:")
completed_df["Age"].describe()


Estadísticas descriptivas de los datos completados de edad:


count    891.000000
mean      29.863791
std       13.389283
min        0.420000
25%       22.000000
50%       29.000000
75%       37.000000
max       80.000000
Name: Age, dtype: float64

#### Valores atípicos (Outliers)
Si bien el dataset presenta valores atípicos en las columnas **Age** y **Fare**, tal como se observa en la sección de Análisis Exploratorio de Datos, estos no parecen corresponder a errores de registro sino a valores extremos reales que aportan información relevante sobre la variabilidad de los datos.<br/>
Por esta razón, **no se eliminarán ni se modificarán**, ya que podrían representar situaciones significativas dentro del contexto del problema.

### Codificación de las variables categóricas

In [29]:
# 1. Codificación binaria para Sex
completed_df["Sex"] = completed_df["Sex"].map({"male": 0, "female": 1})

# 2. Identificar categóricas a one-hot encode
cat_cols = ["Embarked", "Pclass"]

# Asegurar Pclass como categoría
completed_df["Pclass"] = completed_df["Pclass"].astype("category")

# 3. Crear variables dummies
completed_df = pd.get_dummies(completed_df, columns=cat_cols, drop_first=True)

print("Columnas después de codificar:")
print(completed_df.columns)


Columnas después de codificar:
Index(['Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize',
       'IsAlone', 'Embarked_Q', 'Embarked_S', 'Pclass_2', 'Pclass_3'],
      dtype='object')


### Reducción de dimensión (PCA)

El dataset del Titanic tiene pocas columnas relevantes, la dimensionalidad no es alta. Por lo tanto no se aplicará PCA pra reducir su dimensionalidad

### Escalado

In [30]:
cols_to_scale = ["Age", "SibSp", "Parch", "Fare", "FamilySize"]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1) Separar X e y
X = completed_df.drop(columns=["Survived"])
y = completed_df["Survived"]

# 2) Split entrenamiento / validación
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 3) Columnas a escalar
cols_to_scale = ["Age", "SibSp", "Parch", "Fare", "FamilySize"]

# 4) Crear y ajustar el scaler SOLO con el train
scaler = StandardScaler()
scaler.fit(X_train[cols_to_scale])

# 5) Transformar train y valid, dejando el resto igual
X_train_scaled = X_train.copy()
X_valid_scaled = X_valid.copy()

X_train_scaled[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
X_valid_scaled[cols_to_scale] = scaler.transform(X_valid[cols_to_scale])

In [31]:
print(X_train[cols_to_scale].head())
print(X_train_scaled[cols_to_scale].head())
X_train_scaled.head()

      Age  SibSp  Parch      Fare  FamilySize
692  29.6      0      0   56.4958           1
481  32.2      0      0    0.0000           1
527  30.8      0      0  221.7792           1
855  18.0      0      1    9.3500           2
801  31.0      1      1   26.2500           3
          Age     SibSp     Parch      Fare  FamilySize
692 -0.027313 -0.465084 -0.466183  0.513812   -0.556339
481  0.166930 -0.465084 -0.466183 -0.662563   -0.556339
527  0.062338 -0.465084 -0.466183  3.955399   -0.556339
855 -0.893939 -0.465084  0.727782 -0.467874    0.073412
801  0.077279  0.478335  0.727782 -0.115977    0.703162


,Sex,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
692,0,-0.027313,-0.465084,-0.466183,0.513812,-0.556339,1,False,True,False,True
481,0,0.166930,-0.465084,-0.466183,-0.662563,-0.556339,1,False,True,True,False
527,0,0.062338,-0.465084,-0.466183,3.955399,-0.556339,1,False,True,False,False
855,1,-0.893939,-0.465084,0.727782,-0.467874,0.073412,0,False,True,False,True
801,1,0.077279,0.478335,0.727782,-0.115977,0.703162,0,False,True,True,False


### Exportación de datos

In [32]:
import os

os.makedirs("train", exist_ok=True)
os.makedirs("test", exist_ok=True)

X_train_scaled.to_csv("./train/X_train_scaled.csv", index=False)
X_valid_scaled.to_csv("./test/X_valid_scaled.csv", index=False)

y_train.to_csv("./train/y_train.csv", index=False)
y_valid.to_csv("./test/y_valid.csv", index=False)